In [1]:
import pandas as pd
import re
import random
import json
from tqdm.auto import tqdm, trange

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls drive/MyDrive/diploma/data

aligned_chekhov_75.000_sents_09_02.json
aligned_chekhov_chameleon_sents_09_02.json
aligned_chekhov_maria_ivanovna_sents_09_02.json
aligned_chekhov_person_in_case_sents_09_02.json
aligned_chekhov_prishibaev_sents_09_02.json
aligned_chekhov_self-delusion_sents_09_02.json
aligned_chekhov_trifon_sents_09_02.json
aligned_furmanov_chapaev_sents_09_02.json
aligned_gaydar_distant_countries_sents_09_02.json
aligned_gogol_dead_souls_sample_sents_09_02.json
aligned_gorkiy_happiness_sents_09_02.json
aligned_gorkiy_hero_sents_09_02.json
aligned_gorkiy_italian_11_sents_09_02.json
aligned_gorkiy_mother_part2_sents_09_02.json
aligned_gorkiy_mother_sents_09_02.json
aligned_gorkiy_russian_fairy_tales_sents_09_02.json
aligned_konstitution_sents_09_02.json
aligned_land_direct_sents_09_02.json
aligned_news_names_09_02.json
aligned_news_sents_09_02.json
aligned_shchedrin_konyaga_sents_09_02.json
aligned_shchedrin_vilage_fire_sents_09_02.json
all_dicts_data.tsv
all_phrases.tsv
e-mordovia
mdf_mono
moksha_bibl

In [5]:
PARSED_DATA_PATH_PREFIX = 'drive/MyDrive/diploma/data/'

In [6]:
ALIGNED_DATA_PATH_PREFIX = 'drive/MyDrive/diploma/data/'

## Load sentence-parallel data

### bible

In [7]:
bible_df = pd.read_csv(PARSED_DATA_PATH_PREFIX + 'moksha_bible.tsv', sep='\t')
print(bible_df.shape)

bible_df = bible_df.dropna()
print(bible_df.shape)

bible_df = bible_df[~bible_df['mdf'].str.startswith('Глава ')]
print(bible_df.shape)
print(bible_df.columns)

(11650, 4)
(11633, 4)
(11583, 4)
Index(['Unnamed: 0', 'mdf', 'ru', 'source'], dtype='object')


In [8]:
bible_pairs = list(zip(bible_df['mdf'], bible_df['ru']))

print(len(bible_pairs))
print(random.choice(bible_pairs))

11583
('Иисус тусь мархтост. Мзярда Сон ни аф ичкозель кудонц эзда, сядонь оцюнясь кучезень ялганзон азомс Тейнза: Азор, тят томошинда пря. Мон аф кондястян сувафтомс Тонь эсь кудботмозон;', 'Иисус пошел с ними. И когда Он недалеко уже был от дома, сотник прислал к Нему друзей сказать Ему: не трудись, Господи! ибо я недостоин, чтобы Ты вошел под кров мой;')


### words and phrases

In [9]:
word_df = pd.read_csv(PARSED_DATA_PATH_PREFIX + 'all_dicts_data.tsv', sep='\t')

assert not word_df.isna().sum().sum()

word_pairs = list(zip(word_df['mdf'], word_df['ru']))

print(len(word_pairs))
print(random.choice(word_pairs))

3589
('кудста', 'из дома')


In [10]:
phrases_df = pd.read_csv(PARSED_DATA_PATH_PREFIX + 'all_phrases.tsv', sep='\t')

assert not phrases_df.isna().sum().sum()

phrases_pairs = list(zip(phrases_df['mdf'], phrases_df['ru']))

print(len(phrases_pairs))
print(random.choice(phrases_pairs))

726
('Сокаень-видиень оцю куд.', 'Большой крестьянский двор.')


### news

In [12]:
with open(ALIGNED_DATA_PATH_PREFIX + 'aligned_news_names_09_02.json', 'r') as f:
    news_names_df = pd.DataFrame(json.load(f))

assert not news_names_df.isna().sum().sum(), "Missing values detected in news_names"

news_names_pairs = list(zip(news_names_df['mdf'], news_names_df['ru']))

print(len(news_names_pairs))
print(random.choice(news_names_pairs))

4560
('Учителень шить мархта Мордовия Республикань Оцюнять поздравленияц', 'Поздравление Главы Республики Мордовия с Днём учителя')


In [13]:
with open(ALIGNED_DATA_PATH_PREFIX + 'aligned_news_sents_09_02.json', 'r') as f:
    news_sents_df = pd.DataFrame(json.load(f))

assert not news_sents_df.isna().sum().sum(), "Missing values detected in news_sents"

news_sents_pairs = list(zip(news_sents_df['mdf'], news_sents_df['ru']))

print(len(news_sents_pairs))
print(random.choice(news_sents_pairs))

65970
('Работань максыхненди эряви ладямс гибкай график шаба мархта сотрудникненди, башка сятненди, кинь улихть емла классонь тонафнисна.', 'Работодателям стоит по возможности применять гибкий график работы для сотрудников с детьми, особенно у кого есть ученики младших классов.')


### wikisource

In [15]:
document_names = [
    'chekhov_75.000_sents',
    'chekhov_chameleon_sents',
    'chekhov_maria_ivanovna_sents',
    'chekhov_person_in_case_sents',
    'chekhov_prishibaev_sents',
    'chekhov_self-delusion_sents',
    'chekhov_trifon_sents',
    'furmanov_chapaev_sents',
    'gaydar_distant_countries_sents',
    'gogol_dead_souls_sample_sents',
    'gorkiy_happiness_sents',
    'gorkiy_hero_sents',
    'gorkiy_italian_11_sents',
    'gorkiy_mother_sents',
    'gorkiy_mother_part2_sents',
    'gorkiy_russian_fairy_tales_sents',
    'konstitution_sents',
    'land_direct_sents',
    'shchedrin_konyaga_sents',
    'shchedrin_vilage_fire_sents',
]

In [16]:
wikisource_pairs = {}

for doc_name in document_names:
    file_path = ALIGNED_DATA_PATH_PREFIX + f"aligned_{doc_name}_09_02.json"
    with open(file_path, 'r') as f:
        df = pd.DataFrame(json.load(f))

    assert not df.isna().sum().sum(), f"Missing values detected in {doc_name}"

    pairs = list(zip(df['mdf'], df['ru']))

    if doc_name == 'konstitution_sents':
        pairs = [pair for pair in pairs if not pair[1].startswith('Статья')]

    wikisource_pairs[doc_name] = pairs

    print(f"{doc_name}: {len(pairs)} pairs")
    print(random.choice(pairs))

    print()

chekhov_75.000_sents: 130 pairs
('Кодамоль тонь праваце озамс барынятнень мархта стукалкаса налхкома, кда тон содайть, што ся ярмакне аф тоннет, а лиянь?', 'Какое ты право имел садиться с барынями играть в стуколку, если ты знал, что эти деньги не твои, а чужие?')

chekhov_chameleon_sents: 92 pairs
('Пиннясь аф калдяв...', 'Собачонка ничего себе...')

chekhov_maria_ivanovna_sents: 27 pairs
('Аф стама ли произведенияса, конатнень тинць, публикась, кучсесасть эрь шиня пудса минь юмористическяй журналоньконь редакциязост.', 'Не теми ли произведениями, которые вы, читатели, шлете ежедневно пудами в редакции наших газет и журналов?')

chekhov_person_in_case_sents: 186 pairs
('Варенькась тожа ульсь калмамстонза и мзярда лазкснень нолдазь самай калмоти, аварьдсь.', 'Варенька тоже была на похоронах и, когда гроб опускали в могилу, всплакнула.')

chekhov_prishibaev_sents: 84 pairs
('Но, кодак лиссь камерста и няйсь алят конат ащихть толпань-толпань и корхнихьт кати-мезень колга, сонь обуцянц ко

### Union

In [17]:
all_pairs = sorted({
    c for c in
    word_pairs + phrases_pairs + bible_pairs + news_names_pairs + news_sents_pairs + [pair for pairs in wikisource_pairs.values() for pair in pairs]
    if c[0] and c[1]
})
print(len(all_pairs))

87844


In [19]:
with open(ALIGNED_DATA_PATH_PREFIX + 'train_test_splitting/all_pairs.json', 'w') as f:
    json.dump(all_pairs, f, ensure_ascii=False, indent=4)

# Select traindev data

In [20]:
print(len(set(bible_pairs)))

random.seed(1)
bible_devtest = random.sample(sorted(set(bible_pairs)), k=800)

11583


In [21]:
print(len(set(news_sents_pairs)))

random.seed(1)
news_sents_devtest = random.sample(sorted(set(news_sents_pairs)), k=1000)

62308


In [27]:
print(len(set(wikisource_pairs['furmanov_chapaev_sents'])))

random.seed(1)
furmanov_chapaev_devtest = random.sample(sorted(set(wikisource_pairs['furmanov_chapaev_sents'])), k=400)

756


In [28]:
print(len(set(wikisource_pairs['gorkiy_mother_sents'])))

random.seed(1)
gorkiy_mother_devtest = random.sample(sorted(set(wikisource_pairs['gorkiy_mother_sents'])), k=200)

1477


In [29]:
print(len(set(wikisource_pairs['gaydar_distant_countries_sents'])))

random.seed(1)
gaydar_distant_countries_devtest = random.sample(sorted(set(wikisource_pairs['gaydar_distant_countries_sents'])), k=200)

1100


In [30]:
components = {
    'bible': bible_devtest,
    'news_sents': news_sents_devtest,
    'gorkiy_mother': gorkiy_mother_devtest,
    'furmanov_chapaev': furmanov_chapaev_devtest,
    'gaydar_distant_countries': gaydar_distant_countries_devtest
}
dev = {k: v[:len(v)//2] for k, v in components.items()}
test = {k: v[len(v)//2:] for k, v in components.items()}

In [31]:
print(len({s for c in dev.values() for s in c}))
print(len({s for c in test.values() for s in c}))

1300
1300


In [32]:
with open(ALIGNED_DATA_PATH_PREFIX + 'train_test_splitting/dev.json', 'w') as f:
    json.dump(dev, f, ensure_ascii=False, indent=4)

In [33]:
with open(ALIGNED_DATA_PATH_PREFIX + 'train_test_splitting/test.json', 'w') as f:
    json.dump(test, f, ensure_ascii=False, indent=4)

In [34]:
devtest = {s for c in components.values() for s in c}
len(devtest)

2600

In [35]:
train = sorted(set(all_pairs).difference(devtest))
print(len(train))

85244


In [36]:
with open(ALIGNED_DATA_PATH_PREFIX + 'train_test_splitting/train.json', 'w') as f:
    json.dump(train, f, ensure_ascii=False, indent=4)